# Training

In [2]:
import pandas as pd
import numpy as np
import os
import random

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import importlib
import train_models_util
import feature_importance
importlib.reload(train_models_util)
importlib.reload(feature_importance)
from train_models_util import (
    train_xgb_optuna, 
    train_catboost_optuna, 
    train_lgbm_optuna,
    train_best_base_models_from_mlflow,
    train_ensemble, 
    train_neural_network_ensemble,
    train_tree_optuna
)
from evaluate_models_util import evaluate_and_log
from feature_importance import select_important_features, get_top_features_shap

# set all random seeds for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [3]:
EXPERIMENT_NAME = "Fraud-detection-custom-thresholds"

In [4]:
data_folder = 'ieee-fraud-detection-data/processed/'

train = pd.read_csv(f'{data_folder}train_processed.csv')
test = pd.read_csv(f'{data_folder}test_processed.csv')
target = pd.read_csv(f'{data_folder}target.csv')['isFraud']

In [5]:
cat_cols = np.array(train.select_dtypes(include=['object']).columns)
for c in cat_cols:
    if c in train.columns:
        if train[c].isnull().sum()>0 or test[c].isnull().sum()>0:
            train[c].fillna('missing', inplace=True)
            test[c].fillna('missing', inplace=True)
        train[c] = train[c].astype('category')
        test[c] = test[c].astype('category')

## 1. Full Features

### 1.1. XGBoost

In [7]:
model_xgboost_full, best_params_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, hist_df, plot_paths = train_xgb_optuna(train, target, early_stopping_rounds=10)

[I 2025-11-25 17:56:41,467] A new study created in memory with name: xgboost_customloss_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-25 18:01:56,951] Trial 0 finished with value: 0.5528688851642853 and parameters: {'weight_factor': 201.48086673944636, 'learning_rate': 0.1761477943078499, 'max_depth': 14, 'subsample': 0.5832880363511387, 'colsample_bytree': 0.9371982971213139, 'min_child_weight': 0.004706274958444432, 'gamma': 2.978656705624508}. Best is trial 0 with value: 0.5528688851642853.
[I 2025-11-25 18:07:25,500] Trial 1 finished with value: 0.46277973466219635 and parameters: {'weight_factor': 467.51761936437157, 'learning_rate': 0.16957689322270017, 'max_depth': 13, 'subsample': 0.9970464439885749, 'colsample_bytree': 0.5665608827238138, 'min_child_weight': 0.07179838316914401, 'gamma': 4.66234345303025}. Best is trial 1 with value: 0.46277973466219635.
[I 2025-11-25 18:08:21,384] Trial 2 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 666.2898183870409, 'learning_rate': 0.01555796036780099, 'max_depth': 9, 'subsample': 0.5001064315877723, 'colsample_bytree': 0.776805377

[INFO] Best XGBoost params: {'weight_factor': 406.7087130753823, 'learning_rate': 0.06793062060935333, 'max_depth': 11, 'subsample': 0.9388418805360967, 'colsample_bytree': 0.7115663739531671, 'min_child_weight': 0.005153318077608989, 'gamma': 2.6064701071888505}


In [8]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9789104064371599), 'pr_auc': np.float64(0.8939835283899278), 'precision': 0.9673876871880199, 'recall': 0.7033631744495524, 'f1': 0.8145138694312132, 'custom_loss': np.float64(1.038862735801131)}
[INFO] Logged SHAP summary plot.


2025/11/25 21:18:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9789104064371599),
 'pr_auc': np.float64(0.8939835283899278),
 'precision': 0.9673876871880199,
 'recall': 0.7033631744495524,
 'f1': 0.8145138694312132,
 'custom_loss': np.float64(1.038862735801131)}

In [9]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9789104064371599), 'pr_auc': np.float64(0.8939835283899278), 'precision': 0.9141274238227147, 'recall': 0.7984514880232276, 'f1': 0.8523827973653623, 'custom_loss': np.float64(0.7079114031225658)}
[INFO] Logged SHAP summary plot.


2025/11/25 21:19:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9789104064371599),
 'pr_auc': np.float64(0.8939835283899278),
 'precision': 0.9141274238227147,
 'recall': 0.7984514880232276,
 'f1': 0.8523827973653623,
 'custom_loss': np.float64(0.7079114031225658)}

In [10]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9789104064371599), 'pr_auc': np.float64(0.8939835283899278), 'precision': 0.752581664910432, 'recall': 0.8640212920396806, 'f1': 0.8044604640684839, 'custom_loss': np.float64(0.4857757306871677)}
[INFO] Logged SHAP summary plot.


2025/11/25 21:19:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9789104064371599),
 'pr_auc': np.float64(0.8939835283899278),
 'precision': 0.752581664910432,
 'recall': 0.8640212920396806,
 'f1': 0.8044604640684839,
 'custom_loss': np.float64(0.4857757306871677)}

### 1.2. LightGBM

In [11]:
model_lgbm_full, best_params_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, hist_df, plot_paths = train_lgbm_optuna(train, target)

[I 2025-11-25 21:19:24,788] A new study created in memory with name: lgbm_custom_cost_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-25 21:20:22,625] Trial 0 finished with value: 0.8233375419378877 and parameters: {'weight_factor': 481.0034788778232, 'num_leaves': 69, 'max_depth': 15, 'learning_rate': 0.15415735106268225, 'feature_fraction': 0.7134198793587765, 'bagging_fraction': 0.6012476528141174, 'bagging_freq': 5, 'min_child_samples': 48, 'lambda_l1': 5.958943298798557, 'lambda_l2': 0.00019489062151928179}. Best is trial 0 with value: 0.8233375419378877.
[I 2025-11-25 21:21:16,951] Trial 1 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 757.7172685180732, 'num_leaves': 84, 'max_depth': 15, 'learning_rate': 0.03874218973064154, 'feature_fraction': 0.662585883340395, 'bagging_fraction': 0.6892176948186306, 'bagging_freq': 9, 'min_child_samples': 85, 'lambda_l1': 4.278063509094916e-05, 'lambda_l2': 0.0007758493341683748}. Best is trial 0 with value: 0.8233375419378877.
[I 2025-11-25 21:23:08,846] Trial 2 finished with value: 0.5567552207179208 and parameters: {'weight_factor': 

[INFO] Best LightGBM params: {'weight_factor': 119.70658823228621, 'num_leaves': 121, 'max_depth': 20, 'learning_rate': 0.15141577099640155, 'feature_fraction': 0.802855417723744, 'bagging_fraction': 0.9826573945077682, 'bagging_freq': 10, 'min_child_samples': 16, 'lambda_l1': 0.003459889982703883, 'lambda_l2': 7.948719320715972}


In [12]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9798467059331835), 'pr_auc': np.float64(0.9098214217702383), 'precision': 0.9645776566757494, 'recall': 0.7708686184369707, 'f1': 0.85691231845078, 'custom_loss': np.float64(0.8027991329969181)}
[INFO] Logged SHAP summary plot.


2025/11/25 22:11:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9798467059331835),
 'pr_auc': np.float64(0.9098214217702383),
 'precision': 0.9645776566757494,
 'recall': 0.7708686184369707,
 'f1': 0.85691231845078,
 'custom_loss': np.float64(0.8027991329969181)}

In [13]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9798467059331835), 'pr_auc': np.float64(0.9098214217702383), 'precision': 0.9245892809049286, 'recall': 0.8306315025405274, 'f1': 0.87509559010961, 'custom_loss': np.float64(0.5950485995868189)}
[INFO] Logged SHAP summary plot.


2025/11/25 22:12:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9798467059331835),
 'pr_auc': np.float64(0.9098214217702383),
 'precision': 0.9245892809049286,
 'recall': 0.8306315025405274,
 'f1': 0.87509559010961,
 'custom_loss': np.float64(0.5950485995868189)}

In [14]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9798467059331835), 'pr_auc': np.float64(0.9098214217702383), 'precision': 0.8273348519362187, 'recall': 0.8787805468182918, 'f1': 0.8522820603074035, 'custom_loss': np.float64(0.4306058861381109)}
[INFO] Logged SHAP summary plot.


2025/11/25 22:12:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9798467059331835),
 'pr_auc': np.float64(0.9098214217702383),
 'precision': 0.8273348519362187,
 'recall': 0.8787805468182918,
 'f1': 0.8522820603074035,
 'custom_loss': np.float64(0.4306058861381109)}

### 1.3. CatBoost

In [15]:
model_catboost_full, best_params_catboost_full, X_va_catboost_full, y_va_catboost_full, hist_df, plot_paths = train_catboost_optuna(train, target)

[I 2025-11-25 22:12:22,339] A new study created in memory with name: catboost_custom_cost_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-25 22:36:21,632] Trial 0 finished with value: 0.9649854369491365 and parameters: {'weight_factor': 523.5981451218175, 'learning_rate': 0.08179147687289445, 'depth': 16, 'l2_leaf_reg': 7.231789153981489, 'subsample': 0.9738982878476012, 'border_count': 131}. Best is trial 0 with value: 0.9649854369491365.
[I 2025-11-25 22:48:53,133] Trial 1 finished with value: 0.9650087208218796 and parameters: {'weight_factor': 607.1399739192352, 'learning_rate': 0.09918839355667171, 'depth': 13, 'l2_leaf_reg': 2.543243507194481, 'subsample': 0.9484477929205173, 'border_count': 157}. Best is trial 0 with value: 0.9649854369491365.
[I 2025-11-25 22:56:04,286] Trial 2 finished with value: 0.9649536862807844 and parameters: {'weight_factor': 308.28225640245756, 'learning_rate': 0.047111050613887206, 'depth': 10, 'l2_leaf_reg': 4.011569046058383, 'subsample': 0.6856142213135448, 'border_count': 90}. Best is trial 2 with value: 0.9649536862807844.
[I 2025-11-25 23:03:35,143] Trial 3 finished wit

[INFO] Best CatBoost params: {'weight_factor': 974.8758609278207, 'learning_rate': 0.21228137189925653, 'depth': 15, 'l2_leaf_reg': 8.076732668145517, 'subsample': 0.7930965363350582, 'border_count': 239}


In [16]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9792185548859704), 'pr_auc': np.float64(0.9085207718513296), 'precision': 0.9642747151216507, 'recall': 0.7575610936365836, 'f1': 0.8485094850948509, 'custom_loss': np.float64(0.8493582145155282)}
[INFO] Logged SHAP summary plot.


2025/11/26 12:09:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9792185548859704),
 'pr_auc': np.float64(0.9085207718513296),
 'precision': 0.9642747151216507,
 'recall': 0.7575610936365836,
 'f1': 0.8485094850948509,
 'custom_loss': np.float64(0.8493582145155282)}

In [17]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9792185548859704), 'pr_auc': np.float64(0.9085207718513296), 'precision': 0.9323697845650395, 'recall': 0.827244132591338, 'f1': 0.8766666666666667, 'custom_loss': np.float64(0.6066312188844108)}
[INFO] Logged SHAP summary plot.


2025/11/26 12:26:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9792185548859704),
 'pr_auc': np.float64(0.9085207718513296),
 'precision': 0.9323697845650395,
 'recall': 0.827244132591338,
 'f1': 0.8766666666666667,
 'custom_loss': np.float64(0.6066312188844108)}

In [18]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.1)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9792185548859704), 'pr_auc': np.float64(0.9085207718513296), 'precision': 0.8489191729323309, 'recall': 0.874183401887249, 'f1': 0.8613660746215281, 'custom_loss': np.float64(0.44571917228299524)}
[INFO] Logged SHAP summary plot.


2025/11/26 12:44:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9792185548859704),
 'pr_auc': np.float64(0.9085207718513296),
 'precision': 0.8489191729323309,
 'recall': 0.874183401887249,
 'f1': 0.8613660746215281,
 'custom_loss': np.float64(0.44571917228299524)}

## 2. Reduced Features

In [19]:
top_features["xgb"] = get_top_features_shap(model_xgboost_full, train)
top_features["lgbm"] = get_top_features_shap(model_lgbm_full, train)
top_features["catboost"] = get_top_features_shap(model_catboost_full, train)

NameError: name 'top_features' is not defined

In [ ]:
X_reduced, feature_ranking = select_important_features(
    train,
    target,
    top_n=100,                # or None to use cumulative 95%
    use_shap=True,            # optional, slower but more accurate
    remove_correlated=True,
    correlation_threshold=0.95
)

[INFO] Training LightGBM for feature importance estimation...
[INFO] Computing SHAP values for refined importance...
[INFO] Keeping top 100 features by importance.
[INFO] Performing correlation pruning (threshold=0.95)...
[INFO] Dropping 15 correlated features.
[DONE] Selected 85 features out of 407.


### 2.1. XGBoost 

In [47]:
model_xgboost_reduced, best_params_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, hist_df, plot_paths = train_xgb_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 02:07:42,982] A new study created in memory with name: xgboost_customloss_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 02:10:21,382] Trial 0 finished with value: 0.47448512967185286 and parameters: {'weight_factor': 303.03445169252836, 'learning_rate': 0.10083183895081857, 'max_depth': 11, 'subsample': 0.8963608060733238, 'colsample_bytree': 0.6785103891501436, 'min_child_weight': 0.07640426499273617, 'gamma': 4.238768344859967}. Best is trial 0 with value: 0.47448512967185286.
[I 2025-11-20 02:15:17,159] Trial 1 finished with value: 0.47430942625086236 and parameters: {'weight_factor': 384.04306259406184, 'learning_rate': 0.043534070489637824, 'max_depth': 14, 'subsample': 0.7174468047365997, 'colsample_bytree': 0.7170701546516388, 'min_child_weight': 0.45039260296668343, 'gamma': 0.9937966768189904}. Best is trial 1 with value: 0.47430942625086236.
[I 2025-11-20 02:21:49,082] Trial 2 finished with value: 0.438930817423392 and parameters: {'weight_factor': 384.6739597646329, 'learning_rate': 0.029851398336222162, 'max_depth': 11, 'subsample': 0.9780693528395272, 'colsample_bytree': 0.665

[INFO] Best XGBoost params: {'weight_factor': 563.1612148501741, 'learning_rate': 0.016063616494696824, 'max_depth': 15, 'subsample': 0.9610875639100243, 'colsample_bytree': 0.571910755843421, 'min_child_weight': 0.0054253161383116325, 'gamma': 3.032335195843312}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9783492682732146), 'pr_auc': np.float64(0.8923424129990234), 'precision': 0.8718726307808946, 'recall': 0.8347447374788289, 'f1': 0.8529048207663782, 'custom_loss': np.float64(0.5825769634571748)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:15:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9783492682732146), 'pr_auc': np.float64(0.8923424129990234), 'precision': 0.7010428736964078, 'recall': 0.8782966368255505, 'f1': 0.7797229083879282, 'custom_loss': np.float64(0.5825769634571748)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:16:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9783492682732146),
 'pr_auc': np.float64(0.8923424129990234),
 'precision': 0.7010428736964078,
 'recall': 0.8782966368255505,
 'f1': 0.7797229083879282,
 'custom_loss': np.float64(0.5825769634571748)}

### 2.2. LightGBM

In [48]:
model_lgmb_reduced, best_params_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, hist_df, plot_paths = train_lgbm_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 05:16:45,506] A new study created in memory with name: lgbm_custom_cost_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 05:17:31,240] Trial 0 finished with value: 0.6125331219595925 and parameters: {'weight_factor': 988.4441409271524, 'num_leaves': 166, 'max_depth': 20, 'learning_rate': 0.18528910883777497, 'feature_fraction': 0.9538139183876564, 'bagging_fraction': 0.7829489483618187, 'bagging_freq': 5, 'min_child_samples': 65, 'lambda_l1': 3.14241707785604e-05, 'lambda_l2': 0.011144987897729686}. Best is trial 0 with value: 0.6125331219595925.
[I 2025-11-20 05:17:50,230] Trial 1 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 763.9845955234861, 'num_leaves': 115, 'max_depth': 19, 'learning_rate': 0.03601466570668402, 'feature_fraction': 0.9765467245612626, 'bagging_fraction': 0.6274827760378981, 'bagging_freq': 8, 'min_child_samples': 58, 'lambda_l1': 1.4364084634718396e-07, 'lambda_l2': 0.10126313290615058}. Best is trial 0 with value: 0.6125331219595925.
[I 2025-11-20 05:18:13,468] Trial 2 finished with value: 0.8158042208441103 and parameters: {'weight_factor

[INFO] Best LightGBM params: {'weight_factor': 595.0221512419944, 'num_leaves': 76, 'max_depth': 20, 'learning_rate': 0.19236941161792298, 'feature_fraction': 0.879964550074313, 'bagging_fraction': 0.9215867301351228, 'bagging_freq': 4, 'min_child_samples': 84, 'lambda_l1': 8.965510102130184, 'lambda_l2': 1.257085921083824e-06}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9737186890783829), 'pr_auc': np.float64(0.8897176420223434), 'precision': 0.8477306002928258, 'recall': 0.8405516573917251, 'f1': 0.8441258656299356, 'custom_loss': np.float64(0.5632471974802723)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:31:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9737186890783829), 'pr_auc': np.float64(0.8897176420223434), 'precision': 0.7289436477479297, 'recall': 0.8732155819017663, 'f1': 0.7945838837516512, 'custom_loss': np.float64(0.5632471974802723)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:32:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9737186890783829),
 'pr_auc': np.float64(0.8897176420223434),
 'precision': 0.7289436477479297,
 'recall': 0.8732155819017663,
 'f1': 0.7945838837516512,
 'custom_loss': np.float64(0.5632471974802723)}

### 2.3. CatBoost

In [49]:
model_catoost_reduced, best_params_catoost_reduced, X_va_catoost_reduced, y_va_catoost_reduced, hist_df, plot_paths = train_catboost_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_catoost_reduced, X_va_catoost_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_catoost_reduced, X_va_catoost_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 05:32:09,180] A new study created in memory with name: catboost_custom_cost_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 05:35:30,115] Trial 0 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 163.7397266034792, 'learning_rate': 0.03120659802284773, 'depth': 12, 'l2_leaf_reg': 8.181175589517238, 'subsample': 0.6736681243794981, 'border_count': 107}. Best is trial 0 with value: 0.9650108375375834.
[I 2025-11-20 05:42:17,422] Trial 1 finished with value: 0.9649769704447559 and parameters: {'weight_factor': 974.137254461418, 'learning_rate': 0.24457363544379654, 'depth': 16, 'l2_leaf_reg': 4.954780337206101, 'subsample': 0.874814049584014, 'border_count': 54}. Best is trial 1 with value: 0.9649769704447559.
[I 2025-11-20 05:49:22,039] Trial 2 finished with value: 0.9649515700355265 and parameters: {'weight_factor': 829.7542285570263, 'learning_rate': 0.1173314516038542, 'depth': 15, 'l2_leaf_reg': 5.661806434494909, 'subsample': 0.9543732606288229, 'border_count': 97}. Best is trial 2 with value: 0.9649515700355265.
[I 2025-11-20 05:55:44,572] Trial 3 finished with valu

[INFO] Best CatBoost params: {'weight_factor': 826.5125110443701, 'learning_rate': 0.05013487336385276, 'depth': 10, 'l2_leaf_reg': 2.1356969501070076, 'subsample': 0.7829974303923855, 'border_count': 139}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9726674973133655), 'pr_auc': np.float64(0.8668056153521687), 'precision': 0.6505289582212659, 'recall': 0.8778127268328091, 'f1': 0.7472708547888774, 'custom_loss': np.float64(0.44407660785044195)}
[INFO] Logged SHAP summary plot.


2025/11/20 07:45:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9726674973133655), 'pr_auc': np.float64(0.8668056153521687), 'precision': 0.3822463768115942, 'recall': 0.9189450762158239, 'f1': 0.539910441395977, 'custom_loss': np.float64(0.44407660785044195)}
[INFO] Logged SHAP summary plot.


2025/11/20 07:48:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9726674973133655),
 'pr_auc': np.float64(0.8668056153521687),
 'precision': 0.3822463768115942,
 'recall': 0.9189450762158239,
 'f1': 0.539910441395977,
 'custom_loss': np.float64(0.44407660785044195)}

## 3. Ensemble

In [ ]:
base_models, X_tr, X_va, y_tr, y_va, X_test, y_test, best_params_dict = train_best_base_models_from_mlflow(
    train, target, test_size=0.15 ,experiment_name=EXPERIMENT_NAME
)

[INFO] Searching for best xgboost model in experiment 'FRAUD_DETECTION'...
[INFO] Found best xgboost run 2de7afde847a4e978f4d7914a09c9200 with pr_auc=0.9136
[INFO] Retraining xgboost with best parameters...
[INFO] Searching for best catboost model in experiment 'FRAUD_DETECTION'...
[INFO] Found best catboost run 3c8041e4dcf64b9c84c2d5bdd039c60c with pr_auc=0.9082
[INFO] Retraining catboost with best parameters...
[INFO] Searching for best lightgbm model in experiment 'FRAUD_DETECTION'...
[INFO] Found best lightgbm run cd33e5f9d37a4660a9621d0bed29f886 with pr_auc=0.9174
[INFO] Retraining lightgbm with best parameters...


### 3.1. Logistic

In [ ]:
ensemble, base_models, X_meta_va, y_va, X_meta_test, y_test, best_params_log, hist_df, plot_paths = train_ensemble(
    base_models, X_tr, y_tr, X_va, y_va, X_test, y_test, n_trials=50
)

[INFO] Generating meta-features...
[INFO] xgboost validation PR-AUC: 0.8901
[INFO] catboost validation PR-AUC: 0.8128


[I 2025-11-18 03:06:15,837] A new study created in memory with name: ensemble_meta_learner_pr_auc_optimization


[INFO] lightgbm validation PR-AUC: 0.8840
[INFO] Optimizing meta-learner with Optuna...


[Optuna Ensemble Meta-Learner Tuning]:   0%|          | 0/50 [00:00<?, ?trial/s]

[I 2025-11-18 03:06:17,544] Trial 0 finished with value: 0.8968493565501713 and parameters: {'C': 0.004650653818750843, 'class_weight': None, 'solver': 'saga', 'penalty_saga': 'elasticnet', 'l1_ratio': 0.9262925490300581}. Best is trial 0 with value: 0.8968493565501713.
[I 2025-11-18 03:06:18,422] Trial 1 finished with value: 0.8967225113225421 and parameters: {'C': 0.07273320235255908, 'class_weight': 'balanced', 'solver': 'newton-cg', 'penalty_newton-cg': 'l2'}. Best is trial 0 with value: 0.8968493565501713.
[I 2025-11-18 03:06:20,092] Trial 2 finished with value: 0.8969843620283047 and parameters: {'C': 0.05786152184681003, 'class_weight': None, 'solver': 'saga', 'penalty_saga': 'l2'}. Best is trial 2 with value: 0.8969843620283047.
[I 2025-11-18 03:08:18,967] Trial 3 finished with value: 0.7973015325222292 and parameters: {'C': 29.13605449340287, 'class_weight': 'balanced', 'solver': 'sag', 'penalty_sag': None}. Best is trial 2 with value: 0.8969843620283047.
[I 2025-11-18 03:08:1

[INFO] Best meta-learner params: {'C': 0.21957921937861558, 'class_weight': None, 'solver': 'lbfgs', 'penalty_lbfgs': None}
[INFO] Ensemble validation PR-AUC: 0.8972
[INFO] Ensemble validation ROC-AUC: 0.9778
[INFO] Ensemble test PR-AUC: 0.8934
[INFO] Ensemble test ROC-AUC: 0.9770


In [54]:
evaluate_and_log(ensemble, pd.DataFrame(X_meta_va), y_va, experiment_name=EXPERIMENT_NAME, run_name="Ensemble_from_best_base_models", hp_search_history=hist_df)

[INFO] Logged metrics: {'roc_auc': np.float64(0.977775749485985), 'pr_auc': np.float64(0.8971915104190178), 'precision': 0.9516616314199395, 'recall': 0.7621582385676264, 'f1': 0.8464328899637243, 'custom_loss': np.float64(0.8336437836556372)}
[WARN] SHAP skipped: Model type not yet supported by TreeExplainer: <class 'sklearn.linear_model._logistic.LogisticRegression'>


2025/11/18 03:11:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.977775749485985),
 'pr_auc': np.float64(0.8971915104190178),
 'precision': 0.9516616314199395,
 'recall': 0.7621582385676264,
 'f1': 0.8464328899637243,
 'custom_loss': np.float64(0.8336437836556372)}

### 3.2. NN

In [55]:
nn_model, base_mdls, X_val, y_val, X_test, y_test, params, hist, plots = train_neural_network_ensemble(
    base_models=base_models,
    X_tr=X_tr,
    y_tr=y_tr,
    X_va=X_va,
    y_va=y_va,
    X_test=X_test,
    y_test=y_test,
    n_trials=30,
    epochs=100
)

[INFO] Generating meta-features from base models...
[INFO] xgboost validation custom_loss: 0.8598
[INFO] catboost validation custom_loss: 1.3679


[I 2025-11-18 03:12:01,112] A new study created in memory with name: neural_network_ensemble_optimization


[INFO] lightgbm validation custom_loss: 0.9562
[INFO] Optimizing neural network meta-learner with Optuna...


[Optuna NN Ensemble Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-18 03:16:57,637] Trial 0 finished with value: 0.7586107630304467 and parameters: {'n_layers': 1, 'hidden_size_0': 180, 'dropout_rate': 0.38574414781037836, 'learning_rate': 0.005570027698867698, 'weight_decay': 0.000253232244112943}. Best is trial 0 with value: 0.7586107630304467.
[I 2025-11-18 03:26:37,819] Trial 1 finished with value: 0.7185118704914146 and parameters: {'n_layers': 1, 'hidden_size_0': 38, 'dropout_rate': 0.2716836549595695, 'learning_rate': 0.0007150539132222626, 'weight_decay': 5.689583047950391e-06}. Best is trial 1 with value: 0.7185118704914146.
[I 2025-11-18 03:37:02,296] Trial 2 finished with value: 0.6539946489653538 and parameters: {'n_layers': 2, 'hidden_size_0': 200, 'hidden_size_1': 138, 'dropout_rate': 0.014344713117248387, 'learning_rate': 0.009775444648925805, 'weight_decay': 4.869795488201027e-05}. Best is trial 2 with value: 0.6539946489653538.
[I 2025-11-18 03:42:49,435] Trial 3 finished with value: 0.8218664273376909 and parameters: {'n_l

[INFO] Best neural network params: {'n_layers': 3, 'hidden_size_0': 64, 'hidden_size_1': 149, 'hidden_size_2': 123, 'dropout_rate': 0.1519020286431952, 'learning_rate': 0.006663695072321037, 'weight_decay': 1.8779452673203788e-06}
[INFO] Neural network ensemble validation custom_loss: 0.6392
[INFO] Neural network ensemble validation PR-AUC: 0.8229
[INFO] Neural network ensemble validation ROC-AUC: 0.9408
[INFO] Neural network ensemble test custom_loss: 0.6862
[INFO] Neural network ensemble test PR-AUC: 0.8160
[INFO] Neural network ensemble test ROC-AUC: 0.9344
